In [29]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

In [30]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [31]:
crypto = 'BTC'
currency = 'USD'


start = dt.datetime(2016,1,1)

end = dt.datetime.now()


data = web.DataReader( f'{crypto}-{currency}' , 'yahoo' , start , end)

In [32]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-01,436.246002,427.515015,430.721008,434.334015,3.627890e+07,434.334015
2016-01-02,436.062012,431.869995,434.622009,433.437988,3.009660e+07,433.437988
2016-01-03,433.743011,424.705994,433.578003,430.010986,3.963380e+07,430.010986
2016-01-04,434.516998,429.084015,430.061005,433.091003,3.847750e+07,433.091003
2016-01-05,434.182007,429.675995,433.069000,431.959991,3.452260e+07,431.959991
...,...,...,...,...,...,...
2021-09-09,47031.742188,44344.484375,46396.664062,44883.910156,3.915467e+10,44883.910156
2021-09-10,45969.292969,44818.265625,44869.839844,45201.457031,3.449984e+10,45201.457031
2021-09-11,46364.878906,44790.460938,45206.628906,46063.269531,2.788198e+10,46063.269531


In [33]:
scaler = MinMaxScaler(feature_range=(0,1))

scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [34]:
prediction_days = 60 #train for 60 days to predict the 61st day


x_train , y_train = [] , []

for x in range ( prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days:x, 0])
    y_train.append(scaled_data[x,0])
    
    
x_train , y_train = np.array(x_train) , np.array(y_train)

x_train = np.reshape(x_train , (x_train.shape[0] , x_train.shape[1], 1))



In [35]:
#Creating Neural Networks


model = Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape= (x_train.shape[1] , 1)))

model.add(Dropout(0.2)) #to prevent overfitting

model.add(LSTM(units = 50 , return_sequence = True))

model.add(Dropout(0.2))

model.add(LSTM(units = 50) ) 

model.add(Dropout(0.2))
          
model.Dense(units = 1)

model.compile(optimizer = 'adam' , loss = 'mean_squared_error')

model.fit(x_train, y_train, epochs = 25 , batch_size = 32) 

NotImplementedError: Cannot convert a symbolic Tensor (lstm_3/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported